# **Whole slide analysis of Cell DIVE multiplex images**

## <span style="color: red"> THIS NOTEBOOK ACTS AS A TEMPLATE AND WILL NOT SAVE CHANGES UNLESS YOU SAVE A COPY TO YOUR OWN FOLDER. USE `FILE->DOWNLOAD` IN THE TASKBAR AT THE VERY TOP OF THIS PAGE TO SAVE A COPY OF THE NOTEBOOK. THEN, USING THE SIDEBAR TO THE RIGHT OPEN THE NEW COPY OF THE NOTEBOOK BEFORE YOU GET STARTED (RUNNING ON WSL C: AND D: DRIVES SHOULD BE LOCATED UNDER `drives/c` AND `drives/d`, RESPECTIVELY) </span> 

Run through this notebook **step-by-step** and adjust the code if necessary at any point in the process. The explanation and comments througout the notebook will help guide you through the process. There will also be pointers as to modifications you might want to consider.

## Dependencies
We load all the necessary libraries used for the processing, analysis, clustering and visualisation in this notebook.

In [ ]:
library(geometry)
library(uwot)
library(celldiveUtils)
library(stringr)
library(patchwork)

## Reproducibility

You can set a fixed seed to make the results of this notebook reproducible. Set `seed` to an integer value to fix the seed or to `NULL` for a random seed.

In [ ]:
seed = NULL

## 1. Define data location and setup directory structure

### 1.1 Set data directory

The first step is to set the *data/base directory* `base_dir` that contains a folder called `segmentation` with the segmentation and quantification results. The content of the *base* directory would look something like the example below. We will also save the results and all the pipeline outputs in this folder as well.

```
├── ome_tiff
├── image_data
└── segmentation
    ├── cell_table
    ├── cell_table_no_qc
    ├── deepcell_input
    ├── deepcell_output
    ├── deepcell_output_no_qc
    ├── deepcell_visualization
    └── deepcell_visualization_no_qc
    
```

In order to define the *data/base directory* we have two options:
* set `set_base_dir_method="Manual"` and manually define the data/base directory below via `data_dir=`
* set `set_base_dir_method="Relative"` which assume that this notebook is saved in a `notebook` folder alongside the `ome_tiff` in the same parent directory

The last option `Relative` ensures greatest reproducibility and transferability as the notebook is saved alongside the data and all path are defined relative to it.

**NOTE: In `WSL` the `C:` drive, `D:` drive, etc are mounted and located at `/mnt/c`, `/mnt/d`, etc, respectively.**

In [ ]:
set_base_dir_method <- "Relative"

In [ ]:
if (set_base_dir_method == "Relative") {
    notebook_dir <- getwd()
    data_dir <- fs::path_dir(notebook_dir)
} else if (set_base_dir_method == "Manual") {
    data_dir = "/path/to/data/directory"
}

In [ ]:
# set the base directory
base_dir = data_dir

### 1.2 Setup intermediate and results paths

Next, we create results directories to save the plots, etc. such that we have the following folder structure in the `base directory`:

```
├── ome_tiff
├── image_data
├── segmentation
└── analysis
    
```

Here, we set all the directory names to be created.

In [ ]:
results_dir = fs::path_join(c(base_dir, "analysis"))
cell_table_dir = fs::path_join(c(base_dir, "segmentation","cell_table_no_qc"))
cell_table_dir_qc = fs::path_join(c(base_dir, "segmentation", "cell_table"))

Then, we check if the directories exist and if not create them.

In [ ]:
for (directory in c(results_dir)) {
    dir.create(directory,showWarnings = FALSE)
}

## 2. Loading segmented and quantified marker expression data table

First, we need to define if we want to use the size normalized ("normalized") or arcsinh transformed data ("transformed") for the analysis. Please, set the `input_data` variable below accordingly.

In [ ]:
input_data = "normalized" #or "transformed"

In [ ]:
if (input_data == "normalized") {
    file_pattern = ".*_normalized_"
} else if (input_data == "transformed") {
    file_pattern = ".*_arcsinh_transformed_"
}

In [ ]:
cell_table_files = list.files(cell_table_dir_qc, pattern = file_pattern)
tissue_names  = stringr::str_match(cell_table_files,str_glue("cell_table_{file_pattern}(.*?).csv"))[,2]
cell_tables = setNames(cell_table_files, tissue_names)

for(j in 1:length(cell_tables)){
    print(cell_tables[j])
}

Here, we can filter out any slides that we want to exclude from the analysis by subsetting the cell_tables named vector accordingly:

In [ ]:
cell_tables <- cell_tables[1:length(cell_tables)] ## cell_tables[1:2] to only analyse the first two slides 

The slides we will be taking forward for analysis are the following:

In [ ]:
for(j in 1:length(cell_tables)){
    print(cell_tables[j])
}

Next, we combine all the quantified marker expression tables into a common data table and remove any channels that don't occur in all samples. We also apply and offset to each slide to make visualisation of all slides in the same plot more straightforward:

In [ ]:
cells <- list()

cells$metadata <-purrr::map(tissue_names, function(tissue_name) {
    data.table::fread(fs::path_join(c(cell_table_dir_qc,cell_tables[tissue_name]))) %>% 
        dplyr::rename(x = 'centroid-1', y='centroid-0', CellNum=label) %>%
        dplyr::mutate(CellID = paste0('Cell', CellNum, '_', tissue_name)) %>%
        dplyr::mutate(LibraryID = tissue_name)  %>%
        dplyr::select(CellID, LibraryID, x, y, cell_size, DAPI_INIT,DAPI_FINAL, everything()) %>% 
        data.frame()
}) %>% 
    dplyr::bind_rows() %>%
    dplyr::select_if(~ !any(is.na(.))) %>% 
    dplyr::mutate(x_img = x, y_img = y)

y_offset = 0
for(j in unique(cells$metadata$LibraryID)){
    
    cells$metadata <- cells$metadata %>%
        dplyr::mutate(y = dplyr::case_when(
            LibraryID == j ~ y_img + y_offset+1000,
        .default =y))
    
    y_offset <- y_offset +  max((dplyr::filter(cells$metadata, LibraryID == j))$y_img)
    }

Finally, lets visualise the data table,

In [ ]:
cells$metadata %>% 
    split(.$LibraryID) %>% 
    purrr::imap(function(.SD, .name) {
        .SD %>% dplyr::arrange(-y) %>% head(3)
    })

In [ ]:
head(cells$metadata)

In [ ]:
table(cells$metadata$LibraryID)

the segmented cells

In [ ]:
celldiveUtils:::do_scatter(data.frame(x = cells$metadata$y, y = -cells$metadata$x), cells$metadata, 'LibraryID')

and their abundance for each slide.

In [ ]:
data.table::data.table(cells$metadata)[, .N, by = LibraryID] %>% 
    ggplot2::ggplot(ggplot2::aes(LibraryID, N)) + 
        ggplot2::geom_bar(stat = 'identity') + 
        ggplot2::coord_flip() + 
        ggplot2::theme_bw(base_size = 14)

## 3. Filter and process data prior to analysis

First, we separate out the meta/feature data of the segmented cells from the marker intensity values:

In [ ]:
metadata_columns <- c("LibraryID","x","y","cell_size","CellNum", "area", "eccentricity", "major_axis_length", "minor_axis_length", "perimeter", "convex_area", "equivalent_diameter", "major_minor_axis_ratio", "perim_square_over_area", "major_axis_equiv_diam_ratio", "convex_hull_resid", "centroid_dif", "num_concavities", "fov", "mask_type", "x_img", "y_img")
channels_keep <- setdiff(colnames(cells$metadata),metadata_columns)
cells$intensity <- cells$metadata[, channels_keep]
cells$intensity <- cells$intensity %>% tibble::column_to_rownames(var = "CellID")
cells$metadata <- cells$metadata[, c("CellID", metadata_columns)]

Next, we can set low quality channels (`low_quality_channels`) that should be removed ahead of the analysis in the following section. By default only the `'DAPI_INIT'` and  `'DAPI_FINAL'` channels are removed.

In [ ]:
low_quality_channels <- c('PSTAT1', 'CD11C', 'CD34', 'PANCADHERIN', 'CDH11', 'DAPI_INIT', 'DAPI_FINAL', 'MERTK') ## c('DAPI_INIT', 'DAPI_FINAL') ## c('PSTAT1', 'CD11C', 'CD34', 'PANCADHERIN', 'CDH11', 'DAPI_INIT', 'DAPI_FINAL', 'MERTK')

In [ ]:
channels_keep <- setdiff(colnames(cells$intensity), low_quality_channels) ## these are very low quality 
cells$intensity <- cells$intensity[, channels_keep]

After filtering out the low quality channels we perform rescaling of the data.

In [ ]:
#viz channels (density) - the do_scale function has been altered within utils_celldive.r to account for the fact our cells$intensity has already been normalised
cells <- celldiveUtils::do_scale(cells, z_thresh=3, within_batch=TRUE)

Now, we can visualize the data post filtering and scaling:

In [ ]:
cells$z %>% 
    data.frame() %>% 
    tibble::rownames_to_column('CellID') %>% 
    cbind(LibraryID = cells$metadata$LibraryID) %>% 
    tidyr::gather(key, val, all_of(colnames(cells$intensity))) %>% 
    ggplot2::ggplot(ggplot2::aes(val, key, color = LibraryID)) + 
        ggridges::geom_density_ridges2(fill = NA)

In [ ]:
cells$intensity %>% 
    tibble::rownames_to_column('CellID') %>% 
    cbind(LibraryID = cells$metadata$LibraryID) %>% 
    tidyr::gather(key, val, all_of(colnames(cells$intensity))) %>% 
    ggplot2::ggplot(ggplot2::aes(log1p(val), key, color = LibraryID)) + 
        ggridges::geom_density_ridges2(fill = NA)

In [ ]:
#viz intensities (space)
features_plot <- colnames(cells$intensity)
suppressWarnings({
    celldiveUtils::plotFeatures(
        t(cells$z), 
        data.frame(x = cells$metadata$y, y = -cells$metadata$x), 
        features_plot, 
        nrow = floor(sqrt(length(features_plot))),
        no_guide = TRUE
    )    
})

Lastly, lets create spots by pooling information of neighboring cells:

In [ ]:
spots <- celldiveUtils::make_spots(cells)
spots <- celldiveUtils::do_scale(spots, z_thresh=3, within_batch=TRUE)

### 3.2 Cache processed data to the hardrive

In order to save ourselves time in the future, we save and cache the above processed data onto the harddrive. If you have performed the above steps previously you can load `cells.rds` and `spots.rds` at this point in the notebook to proceed. 

In [ ]:
saveRDS(cells, fs::path_join(c(results_dir, 'cells.rds')))

In [ ]:
saveRDS(spots, fs::path_join(c(results_dir, 'spots.rds')))

## 4. Performing harmonisation, dimension reduction and clustering

In this section, we perform data integration using `Harmony`, dimension reduction by `UMAP` and clustering using `Louvain` community detection. We do this both for the spot-based as well as cell-based data.

### 4.1 Spots

We start with the spot data and perform both dimension reduction as well as data integration:

In [ ]:
system.time({
    set.seed(1234)
    spots <- spots %>% 
        # celldiveUtils::do_norm() %>% ## is this step redundant
        # celldiveUtils::do_scale(3, TRUE) %>% # can we do this once and don't repeat it here
        celldiveUtils::do_pca() %>% 
        celldiveUtils::do_umap('V', 'U0', seed=seed, n_sgd_threads = 1, batch=TRUE) %>% 
        celldiveUtils::harmonize() %>% 
        celldiveUtils::do_umap('H', 'U1', seed=seed, n_sgd_threads = 1, batch=TRUE) %>% 
        identity()
})

Next, we visualize the embedding representation following the dimension reduction in the previous step.

In [ ]:
celldiveUtils::fig.size(5, 15)
(
    celldiveUtils::do_scatter(spots$U1$embedding, spots$metadata, 'LibraryID') | 
    celldiveUtils::do_scatter(spots$U1$embedding, spots$metadata, 'LibraryID', dplyr::quo(LibraryID)) 
) + 
patchwork::plot_layout(widths = c(1, 3))

In [ ]:
pdf(fs::path_join(c(results_dir, "features_spots_clustering_umap.pdf")), height=16, width=24)
features_plot <- colnames(spots$z)
suppressWarnings({
    celldiveUtils::plotFeatures(
        t(spots$z), 
        spots$U1$embedding, 
        features_plot, 
        nrow = floor(sqrt(length(features_plot))),
        no_guide = TRUE)    
})
dev.off()

Following this, we perform community detection and clustering using the Louvain algorithm. We need to pass a resolution parameter to the Louvain algorithm. This parameter efffects the size of the identified clusters, e.g. a smaller resolution leads to more but smaller identified clusters and larger resolution on the other hand leads to larger and fewer identified clusters. You can set multiple resolution using the `louvain_clustering_resolution_spots` variable. By default 4 different resolutions are set `c(.1, .4, .8, 1.2)`. Please, adjust this variable according to your data.

In [ ]:
louvain_clustering_resolution_spots <- c(.1, .4, .8, 1.2)

In [ ]:
system.time({
    spots <- spots %>% 
        celldiveUtils::do_louvain('U1', louvain_clustering_resolution_spots) %>% 
        celldiveUtils::do_markers()
})

Next, we plot the results of the clustering step on top of the UMAP.

In [ ]:
## Only plot up to 33 colors 
clusters_plot <- names(which(spots$Clusters %>% map(table) %>% map_int(length) <= length(celldiveUtils:::colors_overload)))

pdf(fs::path_join(c(results_dir, "spots_clustering_umap.pdf")), height=30, width=30)
celldiveUtils::fig.size(4 * length(clusters_plot), 15)
map(clusters_plot, function(.name) {
    celldiveUtils::do_scatter(spots$U1$embedding, spots$Clusters, .name) |
    celldiveUtils::do_scatter(data.frame(x = spots$metadata$y, y = -spots$metadata$x), spots$Clusters, .name, do_labels = FALSE)
}) %>% 
    reduce(`/`)
dev.off()

Lastly, we cache the spots data post clustering to harddrive:

In [ ]:
saveRDS(spots, fs::path_join(c(results_dir, 'spots_post_clustering.rds')))

### 4.2 Cells

Analogous to the previous section, we perform both dimension reduction as well as data integration on the cell-based data:

In [ ]:
system.time({
    set.seed(1234)
    cells <- cells %>% 
        # celldiveUtils::do_norm() %>% ## is redundant
        # celldiveUtils::do_scale(3, TRUE) %>% ## is redundant
        celldiveUtils::do_pca() %>% 
        celldiveUtils::do_umap('V', 'U0', seed=seed, n_sgd_threads = 1, batch=TRUE) %>% 
        celldiveUtils::harmonize() %>% 
        celldiveUtils::do_umap('H', 'U1', seed=seed, n_sgd_threads = 1, batch=TRUE) %>% 
        identity()
})

Next, we visualize the embedding representation following the dimension reduction in the previous step.

In [ ]:
celldiveUtils::fig.size(5, 15)
with(cells, {
    (
        celldiveUtils::do_scatter(U1$embedding, metadata, 'LibraryID') | 
        celldiveUtils::do_scatter(U1$embedding, metadata, 'LibraryID', dplyr::quo(LibraryID)) 
    ) + 
    plot_layout(widths = c(1, 3))
})

In [ ]:
pdf(fs::path_join(c(results_dir, "features_cells_clustering_umap.pdf")), height=16, width=24)
celldiveUtils::fig.size(16, 24)
features_plot <- colnames(cells$z)
suppressWarnings({
    celldiveUtils::plotFeatures(
        t(cells$z), 
        cells$U1$embedding, 
        features_plot, 
        nrow = floor(sqrt(length(features_plot))),
        no_guide = TRUE
    )    
})
dev.off()

Following this, we perform community detection and clustering using the Louvain algorithm. We need to pass a resolution parameter to the Louvain algorithm. This parameter efffects the size of the identified clusters, e.g. a smaller resolution leads to more but smaller identified clusters and larger resolution on the other hand leads to larger and fewer identified clusters. You can set multiple resolution using the `louvain_clustering_resolution_spots` variable. By default 4 different resolutions are set `c(.1, .4, .8, 1.2)`. Please, adjust this variable according to your data.

In [ ]:
louvain_clustering_resolution_cells <- c(.1, .4, .8, 1.2)

In [ ]:
system.time({
    cells <- cells %>% 
        celldiveUtils::do_louvain('U1', louvain_clustering_resolution_cells) %>% 
        celldiveUtils::do_markers()
})

Next, we plot the results of the clustering step on top of the UMAP.

In [ ]:
pdf(fs::path_join(c(results_dir, "cells_clustering_umap.pdf")), height=30, width=30)
## Only plot up to 33 colors 
with(cells, {
    clusters_plot <- names(which(Clusters %>% map(table) %>% map_int(length) <= length(celldiveUtils:::colors_overload)))
    celldiveUtils::fig.size(4 * length(clusters_plot), 15)
    map(clusters_plot, function(.name) {
        celldiveUtils::do_scatter(U1$embedding, Clusters, .name) |
        celldiveUtils::do_scatter(data.frame(x = metadata$y, y = -metadata$x), Clusters, .name, do_labels = FALSE)
    }) %>% 
        reduce(`/`)
    
})
dev.off()

Lastly, we cache cells data post clustering to harddrive:

In [ ]:
saveRDS(cells, fs::path_join(c(results_dir, 'cells_post_clustering.rds')))

## 5. Plotting results 

### 5.1 UMAPs

In this section, we are plotting the clusters in context of the UMAPs and overlay various marker expression. This section in particular is extremely specific to your tissue, samples, staining panel and scientific question. Thus, please make sure to correctly adopt it to your data and questions. 

In [ ]:
pdf(fs::path_join(c(results_dir, "post_plot_clusters.pdf")), height=30, width=30)
with(spots, {
    clusters_plot <- names(which(Clusters %>% map(table) %>% map_int(length) <= length(celldiveUtils:::colors_overload)))
    celldiveUtils::fig.size(4 * length(clusters_plot), 15)
    map(clusters_plot, function(.name) {
        celldiveUtils::do_scatter(U1$embedding, Clusters, .name) |
        celldiveUtils::do_scatter(data.frame(x = metadata$y, y = -metadata$x), Clusters, .name, do_labels = FALSE)
    }) %>% 
        reduce(`/`)
    
})
dev.off()

Here, you can choose which resolution of Louvain clustering to plot by setting the `cluster_resolution_plot` variable accordingly. By default it is set to `0.4`.

In [ ]:
cluster_resolution_plot = 0.4

In [ ]:
cluster_resolution_plot_string = paste0('Clust',cluster_resolution_plot)

In [ ]:
pdf(fs::path_join(c(results_dir, "clusters.pdf")), height = 10, width = 10)
with(spots, celldiveUtils::do_scatter(U1$embedding, Clusters, cluster_resolution_plot_string))
dev.off()

In [ ]:
pdf(fs::path_join(c(results_dir, "clusters_individual.pdf")), height = 15, width = 10)
with(spots, {
    celldiveUtils::do_scatter(U1$embedding, Clusters, cluster_resolution_plot_string, dplyr::quo(!!rlang::sym(cluster_resolution_plot_string)), do_labels=FALSE, nrow=6)       
})
dev.off()

The following code snippet demonstrate how we can overlay the expression of various markers on top of the UMAP and clustering results. Please, consider these merely as example and again adapt them to your specific tissue and question. In this first plot we can visualize to different sets of markers side-by-side by defining the variables `marker_set_1` and `marker_set_2` accordingly.

In [ ]:
marker_set_1 <- c('ASMA', 'CD31', 'CD90', 'CD146')
marker_set_2 <- c('CCL19', 'CD3', 'CD4', 'CD45')

In [ ]:
pdf(fs::path_join(c(results_dir, "markers_across_clusters.pdf")), height = 4, width = 15)
celldiveUtils::plotFeatures(t(spots$z), spots$U1$embedding, marker_set_1, nrow = 1,no_guide = TRUE) %>% plot()
celldiveUtils::plotFeatures(t(spots$z), spots$U1$embedding, marker_set_2, nrow = 1,no_guide = TRUE) %>% plot() 
dev.off()

We can also create a comparitive plot for two tissue feature with corresponding markers by defining the named vector `two_marker_comparison` as, e.g. `c(epithelium = "VIM", stroma = "PANCK")` comparing stroma vs epithelium.

In [ ]:
two_marker_comparison <- c(epithelium = "VIM", stroma = "PANCK")

In [ ]:
pdf(fs::path_join(c(results_dir, str_glue("{names(two_marker_comparison[1])}_vs_{names(two_marker_comparison[2])}_across_clusters.pdf"))), height = 4, width = 8)
celldiveUtils::plotFeatures(t(spots$z), spots$U1$embedding, as.vector(two_marker_comparison), nrow = 1, no_guide = TRUE) %>% plot() 
dev.off()

Lastly, we can also visualise expression of selected markers over clusters identified by the different resolution paremeters of the Louvain algorithm by defining the `markers_set_across_cluster_resolutions` below. By default we loop over all resolutions as defined in the previous section.

In [ ]:
markers_set_across_cluster_resolutions <- c('ASMA', 'CD31', 'CD90', 'CD146', 'CCL19', 'CD3', 'CD4', 'CD45', 'VIM', 'PANCK')

In [ ]:
pdf(fs::path_join(c(results_dir, "heatmap_multiple_res.pdf")), height = 4, width = 8)
# for loop over  louvain_clustering_resolution_spots
for(j in cluster_resolution_plot){
    cluster_resolution_tmp = paste0('Clust',j)
    celldiveUtils::plot_heatmap(spots$Markers[[cluster_resolution_tmp]], markers_set_across_cluster_resolutions, TRUE)
}
dev.off()

## 5.2 Niche heatmaps

In this last step, we annotate the clusters into broad niches. The code below is purely illustrative and you need to adjust the cluster indices and corresponding niche annotation to your tissue and staining as well as which cluster resolution to use (`cluster_resolution_niche_annotation`). By default we use the highest resolution as defined previously by `louvain_clustering_resolution_cells`.

In [ ]:
cluster_resolution_niche_annotation = max(louvain_clustering_resolution_cells)

In [ ]:
if ('Niche' %in% colnames(spots$metadata)) 
    spots$metadata$Niche <- NULL

cluster_resolution_niche_annotation_string = paste0('Clust',cluster_resolution_niche_annotation)
spots$metadata$Cluster <- paste0('C', spots$Clusters[[cluster_resolution_niche_annotation_string]])

spots$metadata <- spots$metadata %>% 
    dplyr::mutate(Niche = dplyr::case_when(
        Cluster %in% paste0('C', c(12,1,0,26,8,18)) ~ 'Lympho_Vascular',
        Cluster %in% paste0('C', c(2,5,14,17,16)) ~ 'Perivascular',
        Cluster %in% paste0('C', c(13,32,4,3,27,11,10,30,9,28)) ~ 'Other',
        Cluster %in% paste0('C', c(29,19,23,21,25,31)) ~ 'Epithelial',
        Cluster %in% paste0('C', c(6,22,7,20,15,24)) ~ 'Lymphoid'
    )) %>% 
    dplyr::mutate(Niche_Broad = gsub('\\d+', '', Niche))

Now, we visualize the annotated Niche both as UMAPs and heatmaps.

In [ ]:
pdf(fs::path_join(c(results_dir, "niches_umap.pdf")), width=8, height=8)
celldiveUtils::fig.size(8, 12)
with(spots, {
    celldiveUtils::do_scatter(U1$embedding, metadata, 'Niche', do_labels = TRUE)    
})
dev.off()

In [ ]:
pdf(fs::path_join(c(results_dir, "niches.pdf")), width=12, height=8)
celldiveUtils::fig.size(8, 12)
with(spots, {
    celldiveUtils::do_scatter(
        data.frame(x = metadata$y, y = -metadata$x),
        metadata, 
        'Niche', dplyr::quo(Niche_Broad), 
        do_labels = FALSE, 
        nrow = 2
    )    
})
dev.off()

In [ ]:
pdf(fs::path_join(c(results_dir, "niche_heatmaps.pdf")), width=8, height=10)
celldiveUtils::fig.size(8, 10)
with(spots, presto::wilcoxauc(t(z), metadata$Niche)) %>% 
    celldiveUtils::plot_heatmap(.scale=TRUE)
dev.off()

Finally, we cache and save the spots data post niche annotation to harddrive :

In [ ]:
saveRDS(spots, fs::path_join(c(results_dir, 'spots_post_niche_annotation.rds')))